# 1. Installs, Imports and Settings

In [1]:
#!pip install spacy==3.1.1 #restart runtime after this
#!python -m spacy download en_core_web_sm

import timer
import csv
import pandas as pd
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)

import string
import operator
from itertools import islice
from collections import Counter

#!pip install nltk
from nltk import ngrams
import nltk as nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet');

import spacy
nlp = spacy.load("es_core_news_md")

#!pip install pyenchant
import enchant

import time

#  2. Reading files 

In [2]:
df = pd.read_csv("data/alertas_NNAJ_keywords.csv", sep="|")

print("Size of Data:", len(df))
print()
print("Columns : ", list(df.columns))

Size of Data: 1753

Columns :  ['Filename', 'Text', 'Subtype', 'Type', 'Year', 'Path', 'Departamento', 'NNAJ', 'NNAJ_clean_text', 'NNAJ_keywords', 'NNAJ_keywords2', 'NNAJ_keywords3']


In [4]:
df.head(1)

Filename  \
0  AT N° 003-18 NAR-Cumbitara, Maguí Payán, Policarpa y Roberto Payán.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [18]:
def mention_recruitment_use(text):
    if type(text) != str:
        text = ' '.join(text)
    list_ = ['reclutamiento', 'uso', 'utilización']
    if any(word in text for word in list_):
        return True
    return False

df['is_recruitment'] = df['NNAJ'].apply(mention_recruitment_use)

len(df[df['is_recruitment'] == True])

864

# 3. Functions

In [25]:
def get_topKWDs(dfcolumn):

    data = { 
             'keywords': list(Counter(dfcolumn).keys()), 
             'frequency': list(Counter(dfcolumn).values())
           }

    counterdf = pd.DataFrame(data)
    counterdf = counterdf.groupby('keywords').agg({'frequency' : 'sum'}).sort_values('frequency', ascending = False)
    return counterdf

def get_list(colname, n):
    allcols = df[df['is_recruitment'] == True][colname].sum() #only mentions of recruitment and use
    print(len(df[df['is_recruitment'] == True][colname]))
    if type(allcols) == str:
        allcols = allcols.strip("']['").split("', '")
    return get_topKWDs(allcols)[:n] if n else get_topKWDs(allcols)

In [26]:
list1 = get_list('NNAJ_keywords', None)
list2 = get_list('NNAJ_keywords2', None)
list3 = get_list('NNAJ_keywords3', None)

864
864
864


In [30]:
len(df)

1753

# 4. Trigrams

In [27]:
list3.head(20)

,frequency
keywords,
grupo armado ilegal,1313
uso ilícito nnaj,768
colombiano bienestar familiar,645
parte grupo armado,606
reclutamiento uso ilícito,575
reclutamiento forzado uso,519
reclutamiento uso nnaj,516
prevención reclutamiento uso,465
alerta temprano versión,380


In [31]:
len(list3)

219427

In [34]:
list3[list3.frequency>10].describe()

,frequency
count,1609.000000
mean,31.849596
std,60.845952
min,11.000000
25%,13.000000
50%,17.000000
75%,29.000000
max,1313.000000


In [37]:
list(list3[640:1600].index)

['nnaj encontrar riesgo',
 'bien mueble inmueble',
 'forzado uso parte',
 'educación nacional coordinación',
 'servidor público líder',
 'nnaj habitante municipio',
 'materia prevención atención',
 'grupo poblacional riesgo',
 'indiscriminado artefacto explosivo',
 'riesgo reclutamiento ejecución',
 'área estratégico desempeño',
 'desaparición reclutamiento uso',
 'grupo armado hacer',
 'persona defensora derechos',
 'forzado población civil',
 'urgente población escolarizado',
 'social vulnerable condición',
 'coordinación necesario promover',
 'control territorial social',
 'social colombiano bienestar',
 'desaparición forzado violencia',
 'uso nnaj restricción',
 'violento grupo armado',
 'vulnerabilidad nnaj mujer',
 'múltiple accidente mina',
 'civil habitar municipio',
 'movilidad desplazamiento forzado',
 'temprano urgente población',
 'llamado limpieza social',
 'armado nnaj mujer',
 'defensoría pueblo regional',
 'gobernación nariño alcaldía',
 'armado ilegal grupo',
 'civil h

In [35]:
get_list('NNAJ_keywords', 50)

864


,frequency
keywords,
nnaj,11376
reclutamiento,4406
armado,4340
grupo,3913
uso,3516
población,3340
forzado,3234
riesgo,3089
municipio,2665


In [47]:
dfr = df[df['is_recruitment'] == True]
len(dfr)

864

In [48]:
dfr.columns

Index(['Filename', 'Text', 'Subtype', 'Type', 'Year', 'Path', 'Departamento',
       'NNAJ', 'NNAJ_clean_text', 'NNAJ_keywords', 'NNAJ_keywords2',
       'NNAJ_keywords3', 'is_recruitment'],
      dtype='object')

In [49]:
dfr = dfr[['Filename', 'NNAJ']]
dfr[['estrategia', 'risk_factors', 'prevention']] = ['','','']
dfr.to_csv('data/alertas_reclutamiento_uso_NNAJ.csv', sep="|", index=False)

## 5. Exporting to html

In [34]:
!jupyter nbconvert --to html 9_NNAJ_Report.ipynb

[NbConvertApp] Converting notebook 9_NNAJ_Report.ipynb to html
[NbConvertApp] Writing 603469 bytes to 9_NNAJ_Report.html
